In [ ]:
def transformer_str(k):
    binary_string_final = ""
    for hex_string in k:
        pairs = [hex_string[i:i+2] for i in range(0, len(hex_string), 2)]
        binary_list = [bin(int(pair, 16))[2:].zfill(8) for pair in pairs]
        binary_string = ''.join(binary_list)
        binary_string_final += binary_string
    return binary_string_final

In [ ]:
def entropy_analysis(k):
  #this function will be used to measure the randomness of cipher text this feature
  # might be a little less useful
  prob_0 = k.count('0') / len(k)
  prob_1 = k.count('1') / len(k)

    # Avoid log(0) by ensuring probabilities > 0
  entropy = 0
  if prob_0 > 0:
      entropy -= prob_0 * np.log2(prob_0)
  if prob_1 > 0:
      entropy -= prob_1 * np.log2(prob_1)

  return entropy

In [ ]:
def frequency_test(k):
  #checks the deviation form stability
    E_i = 0.5
    prob_0 = k.count('0') / len(k)
    prob_1 = k.count('1') / len(k)
    chi_squared = (((prob_0 - E_i) ** 2) / E_i) + (((prob_1 - E_i) ** 2) / E_i)
    deviation = chi_squared ** 0.5  # Taking the square root for stability test

    return deviation

In [ ]:
def run_length(k):
  #used to find the A run of length k
  #consists of exactly k identical bits and
  #is bounded before and after with a bit of opposite value.
  runs = [(key, len(list(group))) for key, group in groupby(k)]

    # Separate runs of 0s and 1s
  run_lengths_0 = [length for bit, length in runs if bit == '0']
  run_lengths_1 = [length for bit, length in runs if bit == '1']

    # Calculate mean run lengths, handle empty cases gracefully
  mean_run_0 = np.mean(run_lengths_0) if run_lengths_0 else 0
  mean_run_1 = np.mean(run_lengths_1) if run_lengths_1 else 0

  return [mean_run_0, mean_run_1]

In [ ]:
def hamming_weight(k):
  #counts number of 1
  return k.count('1')

In [ ]:
def bit_transition(segment):
    """
    Count the number of bit transitions (changes from 0 to 1 or 1 to 0) in the binary string.

    Parameters:
        segment (str): Binary string.

    Returns:
        int: Number of transitions.
    """
    # Convert the binary string to a list of integers
    data = list(map(int, segment))

    # Initialize count of transitions
    count = 0

    # Loop through adjacent bits
    for i in range(len(data) - 1):
        count += abs(data[i] - data[i + 1])  # Compute transition

    return count

In [ ]:
def binary_skewness_kurtosis(k):
    # Convert binary string into a list of integers
    data = list(map(int, k))
    n = len(data)
    # Calculate Mean
    mean = sum(data) / n
    # Calculate Median
    sorted_data = sorted(data)
    if n % 2 == 1:  # Odd number of elements
        median = sorted_data[n // 2]
    else:  # Even number of elements
        median = (sorted_data[n // 2 - 1] + sorted_data[n // 2]) / 2
    # Calculate Standard Deviation
    variance = sum((x - mean) ** 2 for x in data) / n
    std_dev = variance ** 0.5
    if std_dev == 0:  # Handle edge case where all elements are identical
        return 0,0
    # Calculate Skewness
    skewness = 3 * (mean - median) / std_dev
    # Calculate Kurtosis
    kurtosis = sum((x - mean) ** 4 for x in data) / (n * std_dev ** 4) - 3

    return skewness, kurtosis


In [ ]:
def autocorrelation(binary_seq, lag):
    # Convert binary string into a list of integers
    data = list(map(int, binary_seq))
    n = len(data)
    if lag >= n:
        return "Lag is too large for the sequence length."
    # Calculate Mean
    mean = sum(data) / n
    # Calculate Variance
    variance = sum((x - mean) ** 2 for x in data) / n
    if variance == 0:  # Handle edge case where all elements are identical
        return 0
    # Calculate Autocorrelation for the given lag
    autocorr = sum((data[t] - mean) * (data[t + lag] - mean) for t in range(n - lag)) / variance

    return autocorr

In [ ]:
def fractal_dimension(binary_seq):
    """
    Compute the fractal dimension of a binary sequence.

    Parameters:
        binary_seq (str): Binary string representing the sequence.

    Returns:
        float: Fractal dimension of the sequence.
    """
    # Convert binary string into a list of integers
    data = list(map(int, binary_seq))
    n = len(data)

    # Define possible scales
    scales = [2**i for i in range(1, int(math.log2(n)) + 1)] if n > 1 else []

    log_r = []
    log_Nr = []

    for r in scales:
        # Divide sequence into chunks of size r
        chunks = [data[i:i + r] for i in range(0, n, r)]

        # Count non-empty chunks (chunks with at least one "1")
        N_r = sum(1 for chunk in chunks if sum(chunk) > 0)

        # Avoid log(0) errors
        if N_r > 0 and r > 0:
            log_r.append(math.log(r))
            log_Nr.append(math.log(N_r))

    # Handle edge cases where log lists are empty
    if len(log_r) < 2 or len(log_Nr) < 2:
        return 0.0  # Cannot compute fractal dimension for insufficient data

    # Calculate slope (Fractal Dimension) using the formula: slope = Δy / Δx
    D = (log_Nr[-1] - log_Nr[0]) / (log_r[-1] - log_r[0])
    return D

In [ ]:
def extract_features(binary_string):
    """
    Extract features from binary ciphertext for training a model.

    Parameters:
        binary_string (str): Binary sequence representing the ciphertext.

    Returns:
        dict: A dictionary containing extracted features.
    """
    import numpy as np

    # Feature: Length of ciphertext
    length = len(binary_string)

    # Feature: Frequency of '1's and '0's
    ones = binary_string.count('1') / length
    zeros = binary_string.count('0') / length

    # Feature: Spectral Density (FFT mean and variance)
    numeric_ciphertext = np.array([int(bit) for bit in binary_string]) * 2 - 1
    fft_result = np.fft.fft(numeric_ciphertext)
    spectral_density = np.abs(fft_result) ** 2
    spectral_mean = np.mean(spectral_density)
    spectral_variance = np.var(spectral_density)

    # Compile features into a dictionary
    features = [
         spectral_mean,
         spectral_variance
    ]

    return features

In [ ]:
def compute_convolution(ciphertext, kernel=[1,-1]):
    """
    Compute the convolution of a ciphertext with a given kernel.

    Parameters:
        ciphertext (list or np.array): Numeric representation of the ciphertext.
        kernel (list or np.array): Convolutional filter or kernel.

    Returns:
        np.array: Convolution result.
    """
    # Convert ciphertext and kernel to numpy arrays for efficient computation
    ciphertext = np.array(ciphertext)
    kernel = np.array(kernel)
    # Lengths of ciphertext and kernel
    ciphertext_len = len(ciphertext)
    kernel_len = len(kernel)
    # Length of the output
    output_len = ciphertext_len - kernel_len + 1
    # Initialize the convolution result
    convolution_result = np.zeros(output_len)
    # Perform the convolution
    for i in range(output_len):
        convolution_result[i] = np.sum(ciphertext[i:i+kernel_len] * kernel)
    return convolution_result

In [ ]:
import zlib

def compute_ciphertext_complexity(ciphertext):
    """
    Compute the ciphertext complexity using zlib compression.

    Parameters:
        ciphertext (str): Ciphertext as a string.

    Returns:
        int: Length of the compressed ciphertext.
    """
    # Compress the ciphertext using zlib
    compressed_ciphertext = zlib.compress(ciphertext.encode())

    # Return the length of the compressed ciphertext
    return len(compressed_ciphertext)

In [ ]:
import pandas as pd
from joblib import Parallel, delayed

def extract_features_for_row(row, segment_sizes):
    """
    Extract features for a single row of the dataset.

    Parameters:
        row (pd.Series): A row from the dataset containing binary ciphertext in column 4.
        segment_sizes (list): List of segment sizes to extract features for.

    Returns:
        dict: A dictionary of features extracted for the row.
    """
    features = {}
    binary_string = row[4]  # Column containing binary ciphertext

    # Extract features for each segment size
    for size in segment_sizes:
        # Extract the segment of the required size
        segment = binary_string[:size]  # Binary segment (first `size` bits)

        # Pad with zeros if the segment is smaller than the required size
        segment = segment.ljust(size, '0')

        # Extract features for the segment
        features[f"entropy_{size}"] = entropy_analysis(segment)
        features[f"frequency_deviation_{size}"] = frequency_test(segment)
        features[f"mean_run_0_{size}"], features[f"mean_run_1_{size}"] = run_length(segment)
        features[f"hamming_weight_{size}"] = hamming_weight(segment)
        features[f"bit_transitions_{size}"] = bit_transition(segment)
        features[f"skewness_{size}"], features[f"kurtosis_{size}"] = binary_skewness_kurtosis(segment)
        features[f"autocorrelation_{size}"] = autocorrelation(segment, lag=4)
        features[f"fractal_dimension_{size}"] = fractal_dimension(segment)
        spectral_features = extract_features(segment)
        features[f"spectral_mean_{size}"] = spectral_features[0]
        features[f"spectral_variance_{size}"] = spectral_features[1]
        convolution_result = compute_convolution([int(bit) for bit in segment])
        features[f"convolution_{size}"] = sum(convolution_result)
        features[f"complexity_{size}"] = compute_ciphertext_complexity(segment)

    return features

def append_features_to_dataset(dataset, segment_sizes=[128,256,512,1024,2048,4096], n_jobs=-1):
    """
    Append features to the dataset for each ciphertext in column 4 using parallelization.

    Parameters:
        dataset (pd.DataFrame): Original dataset containing binary ciphertexts.
        segment_sizes (list): List of segment sizes to extract features for.
        n_jobs (int): Number of parallel jobs (-1 means using all available processors).

    Returns:
        pd.DataFrame: Updated dataset with appended feature columns.
    """
    # Extract features in parallel for each row
    results = Parallel(n_jobs=n_jobs)(
        delayed(extract_features_for_row)(row, segment_sizes) for _, row in dataset.iterrows()
    )

    # Combine the features back into the DataFrame
    features_df = pd.DataFrame(results)
    updated_dataset = pd.concat([dataset.reset_index(drop=True), features_df], axis=1)

    return updated_dataset

In [ ]:
def chi_square_test(data):
    """Perform chi-square test against uniform distribution."""
    if len(data) == 0:
        return 0
    observed = np.bincount(np.frombuffer(data, dtype=np.uint8), minlength=256)
    expected = np.full(256, len(data)/256)
    chi_square = np.sum((observed - expected)**2 / expected)
    return chi_square


In [ ]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed

def chi_square_test(data):
    """Perform chi-square test against uniform distribution."""
    if len(data) == 0:
        return 0
    observed = np.bincount(np.frombuffer(data.encode('utf-8'), dtype=np.uint8), minlength=256)
    expected = np.full(256, len(data) / 256)
    chi_square = np.sum((observed - expected) ** 2 / expected)
    return chi_square

def extract_features_for_row(row, segment_sizes):
    """
    Extract features for a single row of the dataset.

    Parameters:
        row (pd.Series): A row from the dataset containing binary ciphertext in column 4.
        segment_sizes (list): List of segment sizes to extract features for.

    Returns:
        dict: A dictionary of features extracted for the row.
    """
    features = {}
    binary_string = row[4]  # Column containing binary ciphertext

    # Extract features for each segment size
    for size in segment_sizes:
        # Extract the segment of the required size
        segment = binary_string[:size]  # Binary segment (first `size` bits)

        # Pad with zeros if the segment is smaller than the required size
        segment = segment.ljust(size, '0')

        # Extract features for the segment
        features[f"chi_square_{size}"] = chi_square_test(segment)

    return features

def append_features_to_dataset(dataset, segment_sizes=[128, 256, 512, 1024, 2048, 4096], n_jobs=-1):
    """
    Append features to the dataset for each ciphertext in column 4 using parallelization.

    Parameters:
        dataset (pd.DataFrame): Original dataset containing binary ciphertexts.
        segment_sizes (list): List of segment sizes to extract features for.
        n_jobs (int): Number of parallel jobs (-1 means using all available processors).

    Returns:
        pd.DataFrame: Updated dataset with appended feature columns.
    """
    # Extract features in parallel for each row
    results = Parallel(n_jobs=n_jobs)(
        delayed(extract_features_for_row)(row, segment_sizes) for _, row in dataset.iterrows()
    )

    # Combine the features back into the DataFrame
    features_df = pd.DataFrame(results)
    updated_dataset = pd.concat([dataset.reset_index(drop=True), features_df], axis=1)

    return updated_dataset

In [ ]:
import numpy as np
import pywt
import matplotlib.pyplot as plt

def compute_wavelet_transform(ciphertext, wavelet='haar'):
    """
    Compute the discrete wavelet transform (DWT) of a binary ciphertext.

    Parameters:
        ciphertext (str): Binary ciphertext (e.g., "10110").
        wavelet (str): Wavelet type (default is 'haar').

    Returns:
        list: Wavelet coefficients.
    """
    # Convert binary ciphertext to numeric form (1 -> 1, 0 -> -1)
    numeric_ciphertext = np.array([1 if bit == '1' else -1 for bit in ciphertext])

    # Perform Discrete Wavelet Transform (DWT)
    coeffs = pywt.wavedec(numeric_ciphertext, wavelet)

    return coeffs

In [ ]:
import pandas as pd
import numpy as np
import pywt
from joblib import Parallel, delayed

def wavelet_transform_features(data, wavelet='haar', max_level=5):
    """
    Perform wavelet transform and extract features from the coefficients.

    Parameters:
        data (str): Binary ciphertext string.
        wavelet (str): Type of wavelet to use for the transform.
        max_level (int): Maximum level of decomposition.

    Returns:
        dict: Extracted wavelet features.
    """
    if len(data) == 0:
        return {}

    # Convert binary string to numeric values (1 for '1', -1 for '0')
    numeric_data = np.array([1 if bit == '1' else -1 for bit in data])

    # Perform wavelet transform
    coeffs = pywt.wavedec(numeric_data, wavelet, level=max_level)

    # Extract features from each level
    features = {}
    for level, coeff in enumerate(coeffs):
        features[f'wavelet_mean_level_{level}'] = np.mean(coeff)
        features[f'wavelet_std_level_{level}'] = np.std(coeff)
        features[f'wavelet_energy_level_{level}'] = np.sum(np.square(coeff))
        features[f'wavelet_entropy_level_{level}'] = -np.sum(coeff * np.log2(np.abs(coeff + 1e-10)))

    return features

def extract_features_for_row(row, segment_sizes, wavelet='haar', max_level=4):
    """
    Extract wavelet features for a single row of the dataset.

    Parameters:
        row (pd.Series): A row from the dataset containing binary ciphertext in column 4.
        segment_sizes (list): List of segment sizes to extract features for.
        wavelet (str): Type of wavelet to use for the transform.
        max_level (int): Maximum level of decomposition.

    Returns:
        dict: A dictionary of features extracted for the row.
    """
    features = {}
    binary_string = row[4]  # Column containing binary ciphertext

    # Extract features for each segment size
    for size in segment_sizes:
        # Extract the segment of the required size
        segment = binary_string[:size]  # Binary segment (first `size` bits)

        # Pad with zeros if the segment is smaller than the required size
        segment = segment.ljust(size, '0')

        # Extract wavelet features for the segment
        wavelet_features = wavelet_transform_features(segment, wavelet=wavelet, max_level=max_level)
        features.update({f"{key}_{size}": value for key, value in wavelet_features.items()})

    return features

def append_features_to_dataset(dataset, segment_sizes=[128, 256, 512, 1024, 2048, 4096], wavelet='haar', max_level=4, n_jobs=-1):
    """
    Append wavelet transform features to the dataset for each ciphertext in column 4 using parallelization.

    Parameters:
        dataset (pd.DataFrame): Original dataset containing binary ciphertexts.
        segment_sizes (list): List of segment sizes to extract features for.
        wavelet (str): Type of wavelet to use for the transform.
        max_level (int): Maximum level of decomposition.
        n_jobs (int): Number of parallel jobs (-1 means using all available processors).

    Returns:
        pd.DataFrame: Updated dataset with appended feature columns.
    """
    # Extract features in parallel for each row
    results = Parallel(n_jobs=n_jobs)(
        delayed(extract_features_for_row)(row, segment_sizes, wavelet=wavelet, max_level=max_level)
        for _, row in dataset.iterrows()
    )

    # Combine the features back into the DataFrame
    features_df = pd.DataFrame(results)
    updated_dataset = pd.concat([dataset.reset_index(drop=True), features_df], axis=1)

    return updated_dataset


In [ ]:
from scipy.fft import fft
from scipy.stats import entropy

def spectral_entropy(binary_string):
    numeric_data = np.array([1 if bit == '1' else -1 for bit in binary_string])
    fft_values = np.abs(fft(numeric_data))
    normalized_fft = fft_values / np.sum(fft_values)
    return entropy(normalized_fft)

def dominant_frequency_magnitude(binary_string):
    numeric_data = np.array([1 if bit == '1' else -1 for bit in binary_string])
    fft_values = np.abs(fft(numeric_data))
    return np.max(fft_values)

def power_spectral_density_peaks(binary_string, num_peaks=3):
    numeric_data = np.array([1 if bit == '1' else -1 for bit in binary_string])
    fft_values = np.abs(fft(numeric_data)) ** 2
    return sorted(fft_values, reverse=True)[:num_peaks]


In [ ]:
def entropy_gradient(binary_string, block_size):
    entropies = []
    for i in range(0, len(binary_string), block_size):
        block = binary_string[i:i + block_size]
        counts = np.bincount([int(bit) for bit in block], minlength=2)
        probabilities = counts / counts.sum()
        entropies.append(-np.sum(probabilities * np.log2(probabilities + 1e-10)))
    return np.gradient(entropies)


In [ ]:
def adjacent_bit_probability(binary_string):
    transitions = [1 if binary_string[i] != binary_string[i + 1] else 0 for i in range(len(binary_string) - 1)]
    return sum(transitions) / len(transitions)

def bit_pair_frequency(binary_string):
    pairs = [binary_string[i:i + 2] for i in range(len(binary_string) - 1)]
    return {pair: pairs.count(pair) / len(pairs) for pair in ['00', '01', '10', '11']}


In [ ]:
import nolds

def hurst_exponent(binary_string):
    numeric_data = np.array([1 if bit == '1' else -1 for bit in binary_string])
    return nolds.hurst_rs(numeric_data)

def detrended_fluctuation_analysis(binary_string):
    numeric_data = np.array([1 if bit == '1' else -1 for bit in binary_string])
    return nolds.dfa(numeric_data)


In [ ]:
from pyunicorn.timeseries.recurrence_plot import RecurrencePlot

def recurrence_quantification_analysis(binary_string, threshold=0.5):
    numeric_data = np.array([1 if bit == '1' else -1 for bit in binary_string])
    rp = RecurrencePlot(numeric_data, threshold=threshold)
    return rp.recurrence_rate()


In [ ]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed

# Functions for feature extraction


# Extract features for a single row
def extract_features_for_row(row, segment_sizes):
    features = {}
    binary_string = row['4']  # Column containing binary ciphertext

    for size in segment_sizes:
        # Extract the segment of the required size
        segment = binary_string[:size]
        segment = segment.ljust(size, '0')  # Pad with zeros if smaller than size

        # Compute features
        features[f"recurrence_rate_{size}"] = recurrence_quantification_analysis(segment)
        features[f"hurst_exponent_{size}"] = hurst_exponent(segment)
        features[f"dfa_{size}"] = detrended_fluctuation_analysis(segment)
        bit_freq = bit_pair_frequency(segment)
        for pair, freq in bit_freq.items():
            features[f"bit_pair_freq_{pair}_{size}"] = freq
        features[f"entropy_gradient_{size}"] = entropy_gradient(segment)
        features[f"adjacent_bit_probability_{size}"] = adjacent_bit_probability(segment)
        features[f"spectral_entropy_{size}"] = spectral_entropy(segment)
        features[f"dominant_frequency_magnitude_{size}"] = dominant_frequency_magnitude(segment)
        peaks = power_spectral_density_peaks(segment)
        for i, peak in enumerate(peaks):
            features[f"power_spectral_peak_{i+1}_{size}"] = peak

    return features

# Append features to the dataset
def append_features_to_dataset(dataset, segment_sizes=[128, 256, 512, 1024, 2048, 4096], n_jobs=-1):
    results = Parallel(n_jobs=n_jobs)(
        delayed(extract_features_for_row)(row, segment_sizes) for _, row in dataset.iterrows()
    )
    features_df = pd.DataFrame(results)
    updated_dataset = pd.concat([dataset.reset_index(drop=True), features_df], axis=1)
    return updated_dataset


In [27]:
def transformer_str(k):
    binary_string_final = ""
    for hex_string in k:
        pairs = [hex_string[i:i+2] for i in range(0, len(hex_string), 2)]
        binary_list = [bin(int(pair, 16))[2:].zfill(8) for pair in pairs]
        binary_string = ''.join(binary_list)
        binary_string_final += binary_string
    return binary_string_final

def entropy_analysis(k):
  #this function will be used to measure the randomness of cipher text this feature
  # might be a little less useful
  prob_0 = k.count('0') / len(k)
  prob_1 = k.count('1') / len(k)

    # Avoid log(0) by ensuring probabilities > 0
  entropy = 0
  if prob_0 > 0:
      entropy -= prob_0 * np.log2(prob_0)
  if prob_1 > 0:
      entropy -= prob_1 * np.log2(prob_1)

  return entropy

def frequency_test(k):
  #checks the deviation form stability
    E_i = 0.5
    prob_0 = k.count('0') / len(k)
    prob_1 = k.count('1') / len(k)
    chi_squared = (((prob_0 - E_i) ** 2) / E_i) + (((prob_1 - E_i) ** 2) / E_i)
    deviation = chi_squared ** 0.5  # Taking the square root for stability test

    return deviation

def run_length(k):
  #used to find the A run of length k
  #consists of exactly k identical bits and
  #is bounded before and after with a bit of opposite value.
  runs = [(key, len(list(group))) for key, group in groupby(k)]

    # Separate runs of 0s and 1s
  run_lengths_0 = [length for bit, length in runs if bit == '0']
  run_lengths_1 = [length for bit, length in runs if bit == '1']

    # Calculate mean run lengths, handle empty cases gracefully
  mean_run_0 = np.mean(run_lengths_0) if run_lengths_0 else 0
  mean_run_1 = np.mean(run_lengths_1) if run_lengths_1 else 0

  return [mean_run_0, mean_run_1]

def hamming_weight(k):
  #counts number of 1
  return k.count('1')

def bit_transition(segment):
    """
    Count the number of bit transitions (changes from 0 to 1 or 1 to 0) in the binary string.

    Parameters:
        segment (str): Binary string.

    Returns:
        int: Number of transitions.
    """
    # Convert the binary string to a list of integers
    data = list(map(int, segment))

    # Initialize count of transitions
    count = 0

    # Loop through adjacent bits
    for i in range(len(data) - 1):
        count += abs(data[i] - data[i + 1])  # Compute transition

    return count

def binary_skewness_kurtosis(k):
    # Convert binary string into a list of integers
    data = list(map(int, k))
    n = len(data)
    # Calculate Mean
    mean = sum(data) / n
    # Calculate Median
    sorted_data = sorted(data)
    if n % 2 == 1:  # Odd number of elements
        median = sorted_data[n // 2]
    else:  # Even number of elements
        median = (sorted_data[n // 2 - 1] + sorted_data[n // 2]) / 2
    # Calculate Standard Deviation
    variance = sum((x - mean) ** 2 for x in data) / n
    std_dev = variance ** 0.5
    if std_dev == 0:  # Handle edge case where all elements are identical
        return 0,0
    # Calculate Skewness
    skewness = 3 * (mean - median) / std_dev
    # Calculate Kurtosis
    kurtosis = sum((x - mean) ** 4 for x in data) / (n * std_dev ** 4) - 3

    return skewness, kurtosis

def autocorrelation(binary_seq, lag):
    # Convert binary string into a list of integers
    data = list(map(int, binary_seq))
    n = len(data)
    if lag >= n:
        return "Lag is too large for the sequence length."
    # Calculate Mean
    mean = sum(data) / n
    # Calculate Variance
    variance = sum((x - mean) ** 2 for x in data) / n
    if variance == 0:  # Handle edge case where all elements are identical
        return 0
    # Calculate Autocorrelation for the given lag
    autocorr = sum((data[t] - mean) * (data[t + lag] - mean) for t in range(n - lag)) / variance

    return autocorr

def fractal_dimension(binary_seq):
    """
    Compute the fractal dimension of a binary sequence.

    Parameters:
        binary_seq (str): Binary string representing the sequence.

    Returns:
        float: Fractal dimension of the sequence.
    """
    # Convert binary string into a list of integers
    data = list(map(int, binary_seq))
    n = len(data)

    # Define possible scales
    scales = [2**i for i in range(1, int(math.log2(n)) + 1)] if n > 1 else []

    log_r = []
    log_Nr = []

    for r in scales:
        # Divide sequence into chunks of size r
        chunks = [data[i:i + r] for i in range(0, n, r)]

        # Count non-empty chunks (chunks with at least one "1")
        N_r = sum(1 for chunk in chunks if sum(chunk) > 0)

        # Avoid log(0) errors
        if N_r > 0 and r > 0:
            log_r.append(math.log(r))
            log_Nr.append(math.log(N_r))

    # Handle edge cases where log lists are empty
    if len(log_r) < 2 or len(log_Nr) < 2:
        return 0.0  # Cannot compute fractal dimension for insufficient data

    # Calculate slope (Fractal Dimension) using the formula: slope = Δy / Δx
    D = (log_Nr[-1] - log_Nr[0]) / (log_r[-1] - log_r[0])
    return D

def extract_features(binary_string):
    # Feature: Spectral Density (FFT mean and variance)
    numeric_ciphertext = np.array([int(bit) for bit in binary_string]) * 2 - 1
    fft_result = np.fft.fft(numeric_ciphertext)
    spectral_density = np.abs(fft_result) ** 2
    spectral_mean = np.mean(spectral_density)
    spectral_variance = np.var(spectral_density)

    # Pad or truncate spectral features to ensure consistency
    spectral_features = [spectral_mean, spectral_variance]
    desired_length = 2  # Adjust to match your feature length
    return spectral_features[:desired_length] + [0] * (desired_length - len(spectral_features))


def compute_convolution(ciphertext, kernel=[1,-1]):
    """
    Compute the convolution of a ciphertext with a given kernel.

    Parameters:
        ciphertext (list or np.array): Numeric representation of the ciphertext.
        kernel (list or np.array): Convolutional filter or kernel.

    Returns:
        np.array: Convolution result.
    """
    # Convert ciphertext and kernel to numpy arrays for efficient computation
    ciphertext = np.array(ciphertext)
    kernel = np.array(kernel)
    # Lengths of ciphertext and kernel
    ciphertext_len = len(ciphertext)
    kernel_len = len(kernel)
    # Length of the output
    output_len = ciphertext_len - kernel_len + 1
    # Initialize the convolution result
    convolution_result = np.zeros(output_len)
    # Perform the convolution
    for i in range(output_len):
        convolution_result[i] = np.sum(ciphertext[i:i+kernel_len] * kernel)
    return convolution_result

import zlib

def compute_ciphertext_complexity(ciphertext):
    """
    Compute the ciphertext complexity using zlib compression.

    Parameters:
        ciphertext (str): Ciphertext as a string.

    Returns:
        int: Length of the compressed ciphertext.
    """
    # Compress the ciphertext using zlib
    compressed_ciphertext = zlib.compress(ciphertext.encode())

    # Return the length of the compressed ciphertext
    return len(compressed_ciphertext)

def chi_square_test(data):
    """Perform chi-square test against uniform distribution."""
    if len(data) == 0:
        return 0
    # Convert the string to bytes
    data_bytes = data.encode('utf-8')  # Encode the string as bytes
    observed = np.bincount(np.frombuffer(data_bytes, dtype=np.uint8), minlength=256)
    expected = np.full(256, len(data_bytes) / 256)
    chi_square = np.sum((observed - expected)**2 / expected)
    return chi_square


import numpy as np
import pywt
import matplotlib.pyplot as plt

def wavelet_transform_features(data, wavelet='haar', max_level=5):
    """
    Perform wavelet transform and extract features from the coefficients.

    Parameters:
        data (str): Binary ciphertext string.
        wavelet (str): Type of wavelet to use for the transform.
        max_level (int): Maximum level of decomposition.

    Returns:
        dict: Extracted wavelet features.
    """
    if len(data) == 0:
        return {}

    # Convert binary string to numeric values (1 for '1', -1 for '0')
    numeric_data = np.array([1 if bit == '1' else -1 for bit in data])

    # Perform wavelet transform
    coeffs = pywt.wavedec(numeric_data, wavelet, level=max_level)

    # Extract features from each level
    features = {}
    for level, coeff in enumerate(coeffs):
        features[f'wavelet_mean_level_{level}'] = np.mean(coeff)
        features[f'wavelet_std_level_{level}'] = np.std(coeff)
        features[f'wavelet_energy_level_{level}'] = np.sum(np.square(coeff))
        features[f'wavelet_entropy_level_{level}'] = -np.sum(coeff * np.log2(np.abs(coeff + 1e-10)))

    return features

from scipy.fft import fft
from scipy.stats import entropy

def spectral_entropy(binary_string):
    numeric_data = np.array([1 if bit == '1' else -1 for bit in binary_string])
    fft_values = np.abs(fft(numeric_data))
    normalized_fft = fft_values / np.sum(fft_values)
    return entropy(normalized_fft)

def dominant_frequency_magnitude(binary_string):
    numeric_data = np.array([1 if bit == '1' else -1 for bit in binary_string])
    fft_values = np.abs(fft(numeric_data))
    return np.max(fft_values)

def power_spectral_density_peaks(binary_string, num_peaks=3):
    numeric_data = np.array([1 if bit == '1' else -1 for bit in binary_string])
    fft_values = np.abs(fft(numeric_data)) ** 2
    return sorted(fft_values, reverse=True)[:num_peaks]

def entropy_gradient(binary_string, block_size):
    entropies = []
    for i in range(0, len(binary_string), block_size):
        block = binary_string[i:i + block_size]
        counts = np.bincount([int(bit) for bit in block], minlength=2)
        probabilities = counts / counts.sum()
        entropies.append(-np.sum(probabilities * np.log2(probabilities + 1e-10)))
    return np.gradient(entropies)

def adjacent_bit_probability(binary_string):
    transitions = [1 if binary_string[i] != binary_string[i + 1] else 0 for i in range(len(binary_string) - 1)]
    return sum(transitions) / len(transitions)

def bit_pair_frequency(binary_string):
    pairs = [binary_string[i:i + 2] for i in range(len(binary_string) - 1)]
    freq_dict = {pair: pairs.count(pair) / len(pairs) for pair in ['00', '01', '10', '11']}
    return freq_dict  # Return the dictionary, not a list


import nolds

def hurst_exponent(binary_string):
    numeric_data = np.array([1 if bit == '1' else -1 for bit in binary_string])
    return nolds.hurst_rs(numeric_data)

def detrended_fluctuation_analysis(binary_string):
    numeric_data = np.array([1 if bit == '1' else -1 for bit in binary_string])
    return nolds.dfa(numeric_data)

from pyunicorn.timeseries.recurrence_plot import RecurrencePlot

def recurrence_quantification_analysis(binary_string, threshold=0.5):
    numeric_data = np.array([1 if bit == '1' else -1 for bit in binary_string])
    rp = RecurrencePlot(numeric_data, threshold=threshold)
    return rp.recurrence_rate()


import pandas as pd
from joblib import Parallel, delayed

import numpy as np

import numpy as np

def extract_features_as_vectors(row, segment_sizes, wavelet='haar', max_level=5):
    """
    Extract features for a single row of the dataset and store them as vectors (NumPy arrays or lists).
    
    Parameters:
        row (pd.Series): A row from the dataset containing binary ciphertext in column 4.
        segment_sizes (list): List of segment sizes to extract features for.
        wavelet (str): The wavelet type to use for the wavelet transform. Default is 'haar'.
        max_level (int): The maximum level of the wavelet decomposition.
    
    Returns:
        dict: A dictionary of features stored as vectors (NumPy arrays).
    """
    features_by_type = {}
    binary_string = row[4]  # Column containing binary ciphertext

    # Initialize lists to store features for each type
    feature_names = [
        "entropy", "frequency_test", "run_length_0", "run_length_1",
        "hamming_weight", "bit_transition", "skewness", "kurtosis",
        "autocorrelation", "fractal_dimension", "spectral_features",
        "convolution_sum", "ciphertext_complexity", "chi_square_test",
        "wavelet_mean", "wavelet_std", "wavelet_energy", "wavelet_entropy",
        "recurrence_analysis", "hurst_exponent", "detrended_fluctuation",
        "spectral_peaks"
    ]
    for name in feature_names:
        features_by_type[name] = []  # Initialize as empty lists

    # Extract features for each segment size
    for size in segment_sizes:
        # Divide the binary string into chunks of size 32
        num_chunks = len(binary_string) // 32

        for i in range(num_chunks):
            chunk = binary_string[i * 32:(i + 1) * 32]

            # Pad with zeros if the chunk is smaller than 32 bits
            chunk = chunk.ljust(32, '0')

            # Extract features for the chunk and add them to the respective lists
            features_by_type["entropy"].append(entropy_analysis(chunk))
            features_by_type["frequency_test"].append(frequency_test(chunk))
            r0, r1 = run_length(chunk)
            features_by_type["run_length_0"].append(r0)
            features_by_type["run_length_1"].append(r1)
            features_by_type["hamming_weight"].append(hamming_weight(chunk))
            features_by_type["bit_transition"].append(bit_transition(chunk))
            skewness, kurtosis = binary_skewness_kurtosis(chunk)
            features_by_type["skewness"].append(skewness)
            features_by_type["kurtosis"].append(kurtosis)
            features_by_type["autocorrelation"].append(autocorrelation(chunk, lag=4))
            features_by_type["fractal_dimension"].append(fractal_dimension(chunk))
            spectral_features = extract_features(chunk)
            features_by_type["spectral_features"].extend(spectral_features)  # Assuming multiple values
            convolution_result = compute_convolution([int(bit) for bit in chunk])
            features_by_type["convolution_sum"].append(sum(convolution_result))
            features_by_type["ciphertext_complexity"].append(compute_ciphertext_complexity(chunk))
            features_by_type["chi_square_test"].append(chi_square_test(chunk))
            
            # Wavelet features
            wavelet_features = wavelet_transform_features(chunk, wavelet=wavelet, max_level=max_level)
            for level, features in wavelet_features.items():
                features_by_type["wavelet_mean"].append(np.mean(features))
                features_by_type["wavelet_std"].append(np.std(features))
                features_by_type["wavelet_energy"].append(np.sum(np.square(features)))
                features_by_type["wavelet_entropy"].append(-np.sum(features * np.log2(np.abs(features + 1e-10))))
            
            features_by_type["recurrence_analysis"].append(recurrence_quantification_analysis(chunk))
            features_by_type["hurst_exponent"].append(hurst_exponent(chunk))
            features_by_type["detrended_fluctuation"].append(detrended_fluctuation_analysis(chunk))
            
            peaks = power_spectral_density_peaks(chunk)
            features_by_type["spectral_peaks"].extend(peaks)

    # Convert all lists to NumPy arrays for consistency
    for key in features_by_type.keys():
        features_by_type[key] = np.array(features_by_type[key])

    return features_by_type



from joblib import Parallel, delayed
import pandas as pd
import numpy as np

def append_features_to_dataset(dataset, segment_sizes=[8192], n_jobs=-1):
    """
    Append features to the dataset for each ciphertext in column 4 using parallelization.
    Each feature is stored as a vector for all chunks.

    Parameters:
        dataset (pd.DataFrame): Original dataset containing binary ciphertexts.
        segment_sizes (list): List of segment sizes to extract features for.
        n_jobs (int): Number of parallel jobs (-1 means using all available processors).

    Returns:
        pd.DataFrame: Updated dataset with appended feature columns as vectors.
    """
    def process_row(row, segment_sizes):
        # Extract features as vectors for a single row
        return extract_features_as_vectors(row, segment_sizes)

    # Extract features in parallel for each row
    results = Parallel(n_jobs=n_jobs)(
        delayed(process_row)(row, segment_sizes) for _, row in dataset.iterrows()
    )

    # Combine the features back into the DataFrame
    features_df_list = []
    for result in results:
        # Convert feature dictionary into a single-row DataFrame
        feature_row = {key: np.array(value, dtype=object) for key, value in result.items()}
        features_df_list.append(feature_row)

    # Create a DataFrame from the list of feature dictionaries
    features_df = pd.DataFrame(features_df_list)

    # Fill NaN values with empty arrays for consistency
    for col in features_df.columns:
        features_df[col] = features_df[col].apply(lambda x: x if isinstance(x, np.ndarray) else np.array([], dtype=object))

    # Combine the original dataset with the features
    updated_dataset = pd.concat([dataset.reset_index(drop=True), features_df], axis=1)

    return updated_dataset




In [2]:
import pandas as pd

In [3]:
df=pd.read_csv("encrypted copy.csv",header=None)

In [4]:
df.head()

,0,1,2,3
0,BlockCipher,AES-128,1,0ac1d7274c1f90b0f386d76bb84ef47990afc2c90e70c...
1,BlockCipher,Triple-DES,1,fb33c1f018339f01427793223992dc964bc04c064a98c...
2,BlockCipher,Blowfish,1,d3e59d4c6b095083075fe1620fa371fd496e008db1d0b...
3,BlockCipher,CAST-128,1,7d29e9cc5422da3f2a3f69fae4dbdaa7f3ee17ec8bd96...
4,BlockCipher,AES-128,2,a3aa833397ceb8ecb3d564a537928ae1f43da04973783...


In [5]:
new_df=df[:2].copy()

In [6]:
new_df.head()

,0,1,2,3
0,BlockCipher,AES-128,1,0ac1d7274c1f90b0f386d76bb84ef47990afc2c90e70c...
1,BlockCipher,Triple-DES,1,fb33c1f018339f01427793223992dc964bc04c064a98c...


In [7]:
new_df[3] = [k[1:] for k in new_df[3]]


In [8]:
new_df.head()

,0,1,2,3
0,BlockCipher,AES-128,1,0ac1d7274c1f90b0f386d76bb84ef47990afc2c90e70c6...
1,BlockCipher,Triple-DES,1,fb33c1f018339f01427793223992dc964bc04c064a98c8...


In [9]:
new_df[4]=new_df[3].apply(transformer_str)

In [10]:
new_df.head()

,0,1,2,3,4
0,BlockCipher,AES-128,1,0ac1d7274c1f90b0f386d76bb84ef47990afc2c90e70c6...,0000000000001010000011000000000100001101000001...
1,BlockCipher,Triple-DES,1,fb33c1f018339f01427793223992dc964bc04c064a98c8...,0000111100001011000000110000001100001100000000...


In [11]:
from itertools import groupby

In [12]:
import math

In [28]:
updated_df_1=append_features_to_dataset(new_df)

Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculatin

/Users/inviforce/.pyenv/versions/3.8.12/lib/python3.8/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/inviforce/.pyenv/versions/3.8.12/lib/python3.8/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/inviforce/.pyenv/versions/3.8.12/lib/python3.8/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/inviforce/.pyenv/versions/3.8.12/lib/python3.8/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/inviforce/.pyenv/versions/3.8.12/lib/python3.8/site-packages/sklearn/

Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculatin

/Users/inviforce/.pyenv/versions/3.8.12/lib/python3.8/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/inviforce/.pyenv/versions/3.8.12/lib/python3.8/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/inviforce/.pyenv/versions/3.8.12/lib/python3.8/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/inviforce/.pyenv/versions/3.8.12/lib/python3.8/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculatin

...

/Users/inviforce/.pyenv/versions/3.8.12/lib/python3.8/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/inviforce/.pyenv/versions/3.8.12/lib/python3.8/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/inviforce/.pyenv/versions/3.8.12/lib/python3.8/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/inviforce/.pyenv/versions/3.8.12/lib/python3.8/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/inviforce/.pyenv/versions/3.8.12/lib/python3.8/site-packages/sklearn/

Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculatin

/Users/inviforce/.pyenv/versions/3.8.12/lib/python3.8/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/inviforce/.pyenv/versions/3.8.12/lib/python3.8/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/inviforce/.pyenv/versions/3.8.12/lib/python3.8/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/inviforce/.pyenv/versions/3.8.12/lib/python3.8/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/inviforce/.pyenv/versions/3.8.12/lib/python3.8/site-packages/sklearn/

Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculatin

/Users/inviforce/.pyenv/versions/3.8.12/lib/python3.8/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/inviforce/.pyenv/versions/3.8.12/lib/python3.8/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/inviforce/.pyenv/versions/3.8.12/lib/python3.8/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/inviforce/.pyenv/versions/3.8.12/lib/python3.8/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/inviforce/.pyenv/versions/3.8.12/lib/python3.8/site-packages/sklearn/

In [29]:
updated_df_1.head()

,0,1,2,3,4,entropy,frequency_test,run_length_0,run_length_1,hamming_weight,...,ciphertext_complexity,chi_square_test,wavelet_mean,wavelet_std,wavelet_energy,wavelet_entropy,recurrence_analysis,hurst_exponent,detrended_fluctuation,spectral_peaks
0,BlockCipher,AES-128,1,0ac1d7274c1f90b0f386d76bb84ef47990afc2c90e70c6...,0000000000001010000011000000000100001101000001...,"[0.6252624052234231, 0.8960382325345574, 0.811...","[0.6875, 0.375, 0.5, 0.6875, 0.4375, 0.3125, 0...","[6.75, 4.4, 6.0, 5.4, 4.6, 3.5, 4.0, 4.4, 4.8,...","[1.25, 2.0, 2.0, 1.25, 2.25, 1.833333333333333...","[5, 10, 8, 5, 9, 11, 8, 10, 8, 7, 6, 8, 6, 10,...",...,"[18, 26, 22, 20, 22, 24, 24, 23, 21, 22, 18, 2...","[6000.0, 4640.0, 5088.0, 6000.0, 4848.0, 4464....","[-3.8890872965260126, 0.0, 15.125000000000009,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[15.125000000000009, 0.0, 228.76562500000026, ...","[7.620400616309447, -0.0, -59.27280646392256, ...","[0.736328125, 0.5703125, 0.625, 0.736328125, 0...","[0.6690016464754487, 0.8416460051914575, 0.646...","[0.928349728759552, 1.1063741955776951, 0.5484...","[484.0, 39.31370849898477, 39.31370849898477, ..."
1,BlockCipher,Triple-DES,1,fb33c1f018339f01427793223992dc964bc04c064a98c8...,0000111100001011000000110000001100001100000000...,"[0.9283620723948676, 0.7578784625383954, 0.696...","[0.3125, 0.5625, 0.625, 0.5625, 0.5, 0.625, 0....","[4.2, 6.25, 6.5, 6.25, 6.0, 4.333333333333333,...","[2.2, 2.3333333333333335, 1.5, 1.75, 2.0, 1.2,...","[11, 7, 6, 7, 8, 6, 7, 9, 6, 5, 6, 6, 9, 7, 8,...",...,"[22, 22, 18, 21, 21, 21, 20, 25, 23, 21, 22, 2...","[4464.0, 5360.0, 5664.0, 5360.0, 5088.0, 5664....","[-1.7677669529663693, 0.0, 3.1250000000000018,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[3.1250000000000018, 0.0, 9.76562500000001, 2....","[1.4529773237122166, -0.0, -5.137050593190289,...","[0.548828125, 0.658203125, 0.6953125, 0.658203...","[0.9355795900452806, 0.7059727414177284, 0.492...","[1.2244796988559752, 1.1085088135199368, 0.914...","[227.1959594928933, 227.1959594928933, 100.0, ..."


In [30]:
updated_df_1.dtypes

0                        object
1                        object
2                        object
3                        object
4                        object
entropy                  object
frequency_test           object
run_length_0             object
run_length_1             object
hamming_weight           object
bit_transition           object
skewness                 object
kurtosis                 object
autocorrelation          object
fractal_dimension        object
spectral_features        object
convolution_sum          object
ciphertext_complexity    object
chi_square_test          object
wavelet_mean             object
wavelet_std              object
wavelet_energy           object
wavelet_entropy          object
recurrence_analysis      object
hurst_exponent           object
detrended_fluctuation    object
spectral_peaks           object
dtype: object

In [31]:
updated_df_1['autocorrelation'][0]

array([-5.481481481481482, -12.0, -6.666666666666667, ..., -8.8, -12.0,
       -2.857142857142857], dtype=object)

In [32]:
updated_df_1['autocorrelation'][0][0]

-5.481481481481482

In [33]:
updated_df_1.to_json("features.json", orient="records")


In [38]:
df=pd.read_json("features.json")

/var/folders/cv/thg3bsp52jlbdy4ztqz6b6v40000gn/T/ipykernel_2866/3649744850.py:1: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df=pd.read_json("features.json")
/var/folders/cv/thg3bsp52jlbdy4ztqz6b6v40000gn/T/ipykernel_2866/3649744850.py:1: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df=pd.read_json("features.json")
/var/folders/cv/thg3bsp52jlbdy4ztqz6b6v40000gn/T/ipykernel_2866/3649744850.py:1: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is depr

In [39]:
df.head()

,0,1,2,3,4,entropy,frequency_test,run_length_0,run_length_1,hamming_weight,...,ciphertext_complexity,chi_square_test,wavelet_mean,wavelet_std,wavelet_energy,wavelet_entropy,recurrence_analysis,hurst_exponent,detrended_fluctuation,spectral_peaks
0,BlockCipher,AES-128,1,0ac1d7274c1f90b0f386d76bb84ef47990afc2c90e70c6...,inf,"[0.6252624052, 0.8960382325, 0.8112781245, 0.6...","[0.6875, 0.375, 0.5, 0.6875, 0.4375, 0.3125, 0...","[6.75, 4.4, 6.0, 5.4, 4.6, 3.5, 4.0, 4.4, 4.8,...","[1.25, 2.0, 2.0, 1.25, 2.25, 1.833333333300000...","[5, 10, 8, 5, 9, 11, 8, 10, 8, 7, 6, 8, 6, 10,...",...,"[18, 26, 22, 20, 22, 24, 24, 23, 21, 22, 18, 2...","[6000.0, 4640.0, 5088.0, 6000.0, 4848.0, 4464....","[-3.8890872965, 0.0, 15.125, 7.620400616299999...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[15.125, 0.0, 228.765625, 58.070505553, 0.125,...","[7.6204006162999995, 0.0, -59.2728064639, -22....","[0.736328125, 0.5703125, 0.625, 0.736328125, 0...","[0.6690016465, 0.8416460052, 0.6467084822, 0.6...","[0.9283497288, 1.1063741956, 0.5484275397, 0.7...","[484.0, 39.313708499, 39.313708499, 213.823376..."
1,BlockCipher,Triple-DES,1,fb33c1f018339f01427793223992dc964bc04c064a98c8...,inf,"[0.9283620724, 0.7578784625, 0.6962122601, 0.7...","[0.3125, 0.5625, 0.625, 0.5625, 0.5, 0.625, 0....","[4.2, 6.25, 6.5, 6.25, 6.0, 4.3333333333, 3.57...","[2.2, 2.3333333333, 1.5, 1.75, 2.0, 1.2, 1.166...","[11, 7, 6, 7, 8, 6, 7, 9, 6, 5, 6, 6, 9, 7, 8,...",...,"[22, 22, 18, 21, 21, 21, 20, 25, 23, 21, 22, 2...","[4464.0, 5360.0, 5664.0, 5360.0, 5088.0, 5664....","[-1.7677669530000002, 0.0, 3.125, 1.4529773236...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[3.125, 0.0, 9.765625, 2.1111431032, 1.125, 0....","[1.4529773236999999, 0.0, -5.1370505932, -0.78...","[0.548828125, 0.658203125, 0.6953125, 0.658203...","[0.93557959, 0.7059727414, 0.4928991743, 0.585...","[1.2244796989, 1.1085088135, 0.914965031300000...","[227.1959594929, 227.1959594929, 100.0, 324.0,..."


In [40]:
df.head()

,0,1,2,3,4,entropy,frequency_test,run_length_0,run_length_1,hamming_weight,...,ciphertext_complexity,chi_square_test,wavelet_mean,wavelet_std,wavelet_energy,wavelet_entropy,recurrence_analysis,hurst_exponent,detrended_fluctuation,spectral_peaks
0,BlockCipher,AES-128,1,0ac1d7274c1f90b0f386d76bb84ef47990afc2c90e70c6...,inf,"[0.6252624052, 0.8960382325, 0.8112781245, 0.6...","[0.6875, 0.375, 0.5, 0.6875, 0.4375, 0.3125, 0...","[6.75, 4.4, 6.0, 5.4, 4.6, 3.5, 4.0, 4.4, 4.8,...","[1.25, 2.0, 2.0, 1.25, 2.25, 1.833333333300000...","[5, 10, 8, 5, 9, 11, 8, 10, 8, 7, 6, 8, 6, 10,...",...,"[18, 26, 22, 20, 22, 24, 24, 23, 21, 22, 18, 2...","[6000.0, 4640.0, 5088.0, 6000.0, 4848.0, 4464....","[-3.8890872965, 0.0, 15.125, 7.620400616299999...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[15.125, 0.0, 228.765625, 58.070505553, 0.125,...","[7.6204006162999995, 0.0, -59.2728064639, -22....","[0.736328125, 0.5703125, 0.625, 0.736328125, 0...","[0.6690016465, 0.8416460052, 0.6467084822, 0.6...","[0.9283497288, 1.1063741956, 0.5484275397, 0.7...","[484.0, 39.313708499, 39.313708499, 213.823376..."
1,BlockCipher,Triple-DES,1,fb33c1f018339f01427793223992dc964bc04c064a98c8...,inf,"[0.9283620724, 0.7578784625, 0.6962122601, 0.7...","[0.3125, 0.5625, 0.625, 0.5625, 0.5, 0.625, 0....","[4.2, 6.25, 6.5, 6.25, 6.0, 4.3333333333, 3.57...","[2.2, 2.3333333333, 1.5, 1.75, 2.0, 1.2, 1.166...","[11, 7, 6, 7, 8, 6, 7, 9, 6, 5, 6, 6, 9, 7, 8,...",...,"[22, 22, 18, 21, 21, 21, 20, 25, 23, 21, 22, 2...","[4464.0, 5360.0, 5664.0, 5360.0, 5088.0, 5664....","[-1.7677669530000002, 0.0, 3.125, 1.4529773236...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[3.125, 0.0, 9.765625, 2.1111431032, 1.125, 0....","[1.4529773236999999, 0.0, -5.1370505932, -0.78...","[0.548828125, 0.658203125, 0.6953125, 0.658203...","[0.93557959, 0.7059727414, 0.4928991743, 0.585...","[1.2244796989, 1.1085088135, 0.914965031300000...","[227.1959594929, 227.1959594929, 100.0, 324.0,..."


In [2]:
import pandas as pd

In [10]:
df=pd.read_csv("/Users/inviforce/Downloads/VS_CODE(C++,PYTHON,JUPYTER,DEV)/Sih_features_extracted/encrypted copy.csv",header=None)

In [50]:
pip uninstall tensorflow -y


Found existing installation: tensorflow 2.13.0
Uninstalling tensorflow-2.13.0:
  Successfully uninstalled tensorflow-2.13.0
Note: you may need to restart the kernel to use updated packages.


In [51]:
pip install tensorflow

  Using cached tensorflow-2.13.1-cp38-cp38-macosx_12_0_arm64.whl (1.9 kB)
  Using cached tensorflow-2.13.0-cp38-cp38-macosx_12_0_arm64.whl (1.9 kB)
You should consider upgrading via the '/Users/inviforce/.pyenv/versions/3.8.12/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [53]:
pip uninstall tensorflow tensorflow-estimator keras keras-preprocessing -y

Found existing installation: tensorflow 2.13.0
Uninstalling tensorflow-2.13.0:
  Successfully uninstalled tensorflow-2.13.0
Found existing installation: tensorflow-estimator 2.13.0
Uninstalling tensorflow-estimator-2.13.0:
  Successfully uninstalled tensorflow-estimator-2.13.0
Found existing installation: keras 2.13.1
Uninstalling keras-2.13.1:
  Successfully uninstalled keras-2.13.1
Note: you may need to restart the kernel to use updated packages.


In [54]:
pip cache purge

Files removed: 242
Note: you may need to restart the kernel to use updated packages.


In [55]:
pip install tensorflow


     |████████████████████████████████| 1.7 MB 688 kB/s eta 0:00:01
     |████████████████████████████████| 440 kB 773 kB/s eta 0:00:01
You should consider upgrading via the '/Users/inviforce/.pyenv/versions/3.8.12/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("/Users/inviforce/Downloads/VS_CODE(C++,PYTHON,JUPYTER,DEV)/Sih_features_extracted/new_df1_12.csv")

In [3]:
df.head()

,0,1,2,3,algorithm,4,entropy_128,frequency_deviation_128,mean_run_0_128,mean_run_1_128,...,bit_pair_freq_01_4096,bit_pair_freq_10_4096,bit_pair_freq_11_4096,entropy_gradient_mean_4096,entropy_gradient_std_4096,adjacent_bit_probability_4096,spectral_entropy_4096,dominant_frequency_magnitude_4096,power_spectral_peak_mean_4096,power_spectral_peak_std_4096
0,BlockCipher,AES-128,1,0ac1d7274c1f90b0f386d76bb84ef47990afc2c90e70c6...,AES-128_ 1,0000000000001010000011000000000100001101000001...,0.757878,0.562500,5.555556,1.647059,...,0.156288,0.156288,0.094261,0.797594,0.097619,0.312576,8.107188,2044.0,2.577556e+06,1.131639e+06
1,BlockCipher,Triple-DES,1,fb33c1f018339f01427793223992dc964bc04c064a98c8...,Triple-DES_ 1,0000111100001011000000110000001100001100000000...,0.798659,0.515625,6.062500,1.937500,...,0.145543,0.145543,0.093040,0.776186,0.113530,0.291087,8.093398,2142.0,2.659624e+06,1.363683e+06
2,BlockCipher,Blowfish,1,d3e59d4c6b095083075fe1620fa371fd496e008db1d0bf...,Blowfish_ 1,0000110100000011000011100000010100001001000011...,0.785560,0.531250,4.666667,1.428571,...,0.152137,0.151893,0.101099,0.799202,0.105861,0.304029,8.100660,2022.0,2.638095e+06,1.025580e+06
3,BlockCipher,CAST-128,1,7d29e9cc5422da3f2a3f69fae4dbdaa7f3ee17ec8bd96b...,CAST-128_ 1,0000011100001101000000100000100100001110000010...,0.835116,0.468750,4.272727,1.545455,...,0.153114,0.152869,0.094994,0.788247,0.117146,0.305983,8.103084,2064.0,2.608356e+06,1.167956e+06
4,BlockCipher,AES-128,2,a3aa833397ceb8ecb3d564a537928ae1f43da04973783b...,AES-128_ 2,0000101000000011000010100000101000001000000000...,0.835116,0.468750,4.272727,1.619048,...,0.160684,0.160684,0.094017,0.804364,0.094466,0.321368,8.110214,2010.0,2.568522e+06,1.040563e+06


In [6]:
target = df.iloc[:, 1]  # Adjust based on your dataset

In [12]:
df.head()

,0,1,2,3
0,BlockCipher,AES-128,1,0ac1d7274c1f90b0f386d76bb84ef47990afc2c90e70c...
1,BlockCipher,Triple-DES,1,fb33c1f018339f01427793223992dc964bc04c064a98c...
2,BlockCipher,Blowfish,1,d3e59d4c6b095083075fe1620fa371fd496e008db1d0b...
3,BlockCipher,CAST-128,1,7d29e9cc5422da3f2a3f69fae4dbdaa7f3ee17ec8bd96...
4,BlockCipher,AES-128,2,a3aa833397ceb8ecb3d564a537928ae1f43da04973783...


In [11]:
target = df.iloc[:, 1]  # Adjust based on your dataset
features = df.drop(df.columns[[0, 1, 2, 3, 4]], axis=1)

IndexError: index 4 is out of bounds for axis 0 with size 4

In [15]:
# Step 1: Convert all columns to numeric
print("Converting all columns to numeric types...")
features = features.apply(pd.to_numeric, errors='coerce')  # Convert non-numeric values to NaN

# Step 2: Check for infinite values
print("Checking for infinite values in the dataset...")
inf_count = np.isinf(features.values).sum()
print(f"Number of infinite values: {inf_count}")

# Step 3: Replace infinite values with NaN
if inf_count > 0:
    print("Replacing infinite values with NaN...")
    features.replace([np.inf, -np.inf], np.nan, inplace=True)

# Step 4: Check for NaN values
print("Checking for NaN values in each column...")
nan_count = features.isna().sum()
print(nan_count)

# Step 5: Impute missing values
print("Imputing missing values...")
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="mean")
features = pd.DataFrame(imputer.fit_transform(features), columns=features.columns)

# Final check for non-finite values
print("Final check for non-finite values...")
if not np.all(np.isfinite(features.values)):
    raise ValueError("Dataset still contains non-finite values after preprocessing!")

print("Preprocessing complete!")


Converting all columns to numeric types...
Checking for infinite values in the dataset...
Number of infinite values: 0
Checking for NaN values in each column...
4                                    25025
entropy_128                              0
frequency_deviation_128                  0
mean_run_0_128                           0
mean_run_1_128                           0
                                     ...  
adjacent_bit_probability_4096            0
spectral_entropy_4096                    0
dominant_frequency_magnitude_4096        0
power_spectral_peak_mean_4096            0
power_spectral_peak_std_4096             0
Length: 296, dtype: int64
Imputing missing values...


/Users/inviforce/.pyenv/versions/3.8.12/lib/python3.8/site-packages/sklearn/impute/_base.py:558: UserWarning: Skipping features without any observed values: ['4']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


ValueError: Shape of passed values is (25025, 295), indices imply (25025, 296)

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
import xgboost as xgb
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.impute import SimpleImputer


# Replace these column indices with appropriate values for your dataset
target = df.iloc[:, 1]  # Adjust this to your target column
features = df.drop(df.columns[[0, 1, 2, 3, 4]], axis=1)

# Step 2: Handle Invalid Values in Features
print("Checking for invalid values in the dataset...")

# Replace infinity values with NaN
features.replace([np.inf, -np.inf], np.nan, inplace=True)

# Print NaN summary
print("Missing values per column:")
print(features.isna().sum())

# Impute missing values
imputer = SimpleImputer(strategy="mean")
features = pd.DataFrame(imputer.fit_transform(features), columns=features.columns)

# Check for non-finite values after imputation
if not np.all(np.isfinite(features)):
    raise ValueError("Non-finite values remain in the dataset after imputation!")

# Scale features
scaler = StandardScaler()
features = features

# Step 3: Encode Target
encoder = LabelEncoder()
encoded_target = encoder.fit_transform(target)
target_mapping = dict(zip(encoder.classes_, range(len(encoder.classes_))))

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    features, encoded_target, test_size=0.2, random_state=42, stratify=encoded_target
)

# Step 4: Train XGBoost with Cross-Validation
xgb_params = {
    "objective": "multi:softprob",
    "num_class": len(encoder.classes_),
    "eval_metric": "mlogloss",
    "max_depth": 6,
    "learning_rate": 0.1,
    "n_estimators": 100,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "random_state": 42,
}

xgb_model = xgb.XGBClassifier(**xgb_params)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for train_idx, val_idx in cv.split(X_train, y_train):
    X_train_cv, X_val_cv = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_train_cv, y_val_cv = y_train[train_idx], y_train[val_idx]
    xgb_model.fit(
        X_train_cv,
        y_train_cv,
        eval_set=[(X_val_cv, y_val_cv)],
        early_stopping_rounds=10,
        verbose=False,
    )

# Extract features from XGBoost
xgb_train_features = np.array(xgb_model.apply(X_train)).reshape(len(X_train), -1)
xgb_test_features = np.array(xgb_model.apply(X_test)).reshape(len(X_test), -1)

# Step 5: Build Sequential Model
light_bpm = Sequential([
    Dense(256, activation='relu', input_dim=xgb_train_features.shape[1]),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(len(encoder.classes_), activation='softmax')
])

light_bpm.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the light BPM model
history = light_bpm.fit(
    xgb_train_features,
    y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping],
    verbose=1
)

# Step 6: Evaluate the Model
final_predictions = light_bpm.predict(xgb_test_features)
predicted_classes = np.argmax(final_predictions, axis=1)

# Convert predictions back to original labels
decoded_predictions = encoder.inverse_transform(predicted_classes)
true_labels = encoder.inverse_transform(y_test)

print("Classification Report:")
print(classification_report(true_labels, decoded_predictions))

accuracy = accuracy_score(true_labels, decoded_predictions)
print(f"Final Accuracy: {accuracy * 100:.2f}%")

# Step 7: Save Model Artifacts
xgb_model.save_model("robust_xgb_model.json")
light_bpm.save("robust_light_bpm_model.h5")


Checking for invalid values in the dataset...
Missing values per column:
4                                    0
entropy_128                          0
frequency_deviation_128              0
mean_run_0_128                       0
mean_run_1_128                       0
                                    ..
adjacent_bit_probability_4096        0
spectral_entropy_4096                0
dominant_frequency_magnitude_4096    0
power_spectral_peak_mean_4096        0
power_spectral_peak_std_4096         0
Length: 296, dtype: int64


ValueError: Input X contains infinity or a value too large for dtype('float64').